In [55]:
import numpy as np
import copy

class OwlOptimization():
    def __init__(self):
        self.beta = 1.9
        self.epsilon = 1e-32
    
    def compile(self, obj, dimension, lb, ub, population_size = 100, max_iter = 100, optimization = "maximize"):
        self.objective_function = obj
        self.dimension = dimension
        self.lb = lb
        self.ub = ub
        self.max_iter = max_iter
        self.optimization = optimization
        self.population_size = population_size
    
    def __initialize_population(self, owl_num, dim, lb, ub):
        return lb + np.random.uniform(0, 1, size = (owl_num, dim)) * (ub - lb)
            
    # Equ. 28
    def __norm_sound_intensity(self, f_i, owls):
        
        # Find minimun and maximum value
        w_min = np.min(owls)
        b_max = np.max(owls)

        # Return normalization intensity of owl position by equation 4
        return (f_i - w_min) / (b_max - w_min)

    def __change_sound_intensity(self, sound_intensity, distance_info_R):
        if distance_info_R == 0:
            return ((sound_intensity) / (sound_intensity)) + np.random.uniform(0, 1.0, 1)
        else:
            return ((sound_intensity) / (distance_info_R**2)) + np.random.uniform(0, 1.0, 1)

    def __linearly_dec_constant(self, crnt, max_num, range_min = 0, range_max = 1.9):
        return (crnt - range_min) / (max_num-range_min) * range_max

    # Equ. 31
    def __update_owl_position(self, alpha, beta, owl_locations, fittest_owl, intensity_ICi):

        # Generate random number 0 to 1
        pvm = np.random.uniform(0, 1)

        if pvm < 0.5:
            return owl_locations + beta * intensity_ICi * np.absolute(alpha * fittest_owl - owl_locations)
        else:
            return owl_locations - beta * intensity_ICi * np.absolute(alpha * fittest_owl - owl_locations)
        
    def calculate_optimal_value(self):

        # Get Owl with initial locations # np.array([[4, 23, 5, 6], [1, 6, 8, 3], [4, 3, 5, 6]]) # 
        owls = self.__initialize_population(self.population_size, self.dimension, self.lb, self.ub)

        # Print n * d matrix
        # print(f"Matrix for number of owl with search space:\n{owls}\n")

        epsilon = 1e-32

        for iter in range(self.max_iter):

            candidate_evaluations = np.empty([self.population_size])

            #Calculating the output of objective function 
            for i in range(self.population_size):
                candidate_evaluations[i] = self.objective_function(owls[i, :])

            # Calculate the intensity 
            sound_intensity = np.empty([self.population_size])
            for i in range(self.population_size):
                sound_intensity[i] = self.__norm_sound_intensity(candidate_evaluations[i], owls)

            # print(f"candidate_evaluations: {candidate_evaluations}")
            # print(f"sound_intensity: {sound_intensity}")
            
            # Determining the fittest index. The best owl receives max intensity (for maximization problems) as it is more close to vole.
            if self.optimization == "minimize":
                fittest_idx = np.argmax(sound_intensity)
            elif self.optimization == "maximize":
                fittest_idx = np.argmin(sound_intensity)

            fittest_owl = owls[fittest_idx, :]
            
            # Calculates the distance between owl and prey (equ. 29)
            distances = np.empty([self.population_size])
            for i in range(self.population_size):
                distances[i] = np.linalg.norm(owls[i] - fittest_owl)

            # Obtains the change in intensity (eq. 30)
            intensity_change = np.empty([self.population_size])
            for i in range(self.population_size):
                intensity_change[i] = self.__change_sound_intensity(sound_intensity[i], distances[i])
            
            alpha = np.random.uniform(0, 0.5, 1)
            beta = self.beta - ((iter + 1) / self.max_iter) * self.beta # self.__linearly_dec_constant()
            
            for i in range(self.population_size):
                owls[i] = self.__update_owl_position(alpha, beta, owls[i], fittest_owl, intensity_change[i])

        return fittest_owl

In [56]:
# Objective function
def obj_func(x):
    res = 0
    for i in x:
        res += i**2
    return res
    
dimension = 30
lb = np.array([-100]).repeat(dimension)
ub = np.array([100]).repeat(dimension)
max_iter = 100
population_size = 4

model = OwlOptimization()
model.compile(obj_func, dimension, lb, ub, population_size, max_iter, optimization = 'maximize')
fittest_owl = model.calculate_optimal_value()
print(f"\nFianl Fittest Owl: {fittest_owl}\n")
print(f"Sphere Benchmark: {obj_func(fittest_owl)}")


Fianl Fittest Owl: [-5.99662602e-09 -1.79913185e-10 -1.96009823e-10 -5.09983131e-11
 -2.08984501e-10 -3.32716755e-12 -4.60694180e-11 -1.89003196e-11
 -1.00484205e-10 -1.14786906e-08 -3.13482328e-10 -1.95140346e-11
 -2.89892263e-10 -1.60287576e-10 -8.92446354e-11 -1.69043936e-10
 -1.66949197e-11 -2.26906330e-10 -3.21693730e-09 -6.71700004e-09
 -3.82986544e-11 -4.15921912e-11 -8.14271139e-11 -1.70258300e-11
 -2.36484767e-11 -1.50116301e-10 -2.34100113e-10 -3.86870953e-10
 -1.24154903e-10 -1.63254223e-08]

Sphere Benchmark: 4.90385486237005e-16
